In [ ]:
import sys
!{sys.executable} -m pip install pygeodesy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install multiprocess

In [ ]:
import sqlite3
import json
from pygeodesy.sphericalNvector import LatLon, Nvector
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
import math
import timeit
from threading import Thread
from multiprocess import Process, Pool
from itertools import repeat
%matplotlib notebook

In [ ]:
# load in the sqlite file into memory

db_file= "scan_data.sqlite"

# basic setup
con = sqlite3.connect(db_file)
con.row_factory = sqlite3.Row
cur = con.cursor()

# join on the ID, as the base geometry BLOB in the ScanData table is raw binary
# we could convert that to geomtry via a QGIS package for JSON, but joining works just as well
db_rows = con.execute(
    "SELECT ScanData.pkuid, ScanData.data, ScanData.level, \
    idx_ScanData_geometry.xmin, idx_ScanData_geometry.xmax, idx_ScanData_geometry.ymin, idx_ScanData_geometry.ymax \
    FROM ScanData \
    INNER JOIN idx_ScanData_geometry \
    ON ScanData.pkuid=idx_ScanData_geometry.pkid"
)
# turn it into our favourite thing ever, a dictionary :)
db_dict = [dict(row) for row in db_rows]
print(f"Obtained {len(db_dict)} rows from database.")

# ensure the connection is closed
con.close()

In [ ]:
# load in the AP locational data for referencing

# we dont care about the preamble, so just get the features
with open("heuristic_nodes.json", 'r') as f:
    node_data = json.load(f)['features']
    
# will hold the AP locational data for referencing later
node_dict = {}

# set up dictionary with the MAC as the key, and the level and coordinates as values
for entry in node_data:
    data = entry['properties']
    
    # only take Points with a MAC value, i.e. only take APs
    if not data['mac_addres'] is None:
        
        # skim off the last char of the key. still unique. this makes later processing O(1) instead of O(n)
        node_dict[data['mac_addres'][:-1]] = {
            "level": int(data['level']),
            "coordinates": entry['geometry']['coordinates']
        }

# sort based on level
node_dict = dict(sorted(node_dict.items(), key=lambda x:x[1]['level']))        

for mac, data in node_dict.items():
    print(f"MAC {mac} on Level {data['level']} := {data['coordinates']}")

In [ ]:
def compare_networks(data, level):
    
    common_list = []
    
    for scan in data:
        common_scan = []

        for network in scan:
            mac = network['BSSID'][:-1]
            
            # get networks of known location on the same floor
            try:
                entry = node_dict[mac]
                if entry['level'] == level:
                    
                    # ensure the AP hasnt already been added from a similar MAC but same location
                    # same location will break trilateration entirely
                    if not any(net for net in common_scan if net["BSSID"][:-1] == mac):
                        pos = LatLon(entry['coordinates'][0], entry['coordinates'][1])
                        network['coordinates'] = pos
                        del network["SSID"]
                        common_scan.append(network)
            except Exception:
                ...
        
        common_list.append(common_scan)
            
    return common_list

In [ ]:
# turn sqlite file into dictionary, referencing scan data against AP dict to add locations

# the end dict that will hold all data for trilateration
ap_dict = {}

for row in db_dict:
    
    # point is a circle defined by bounding box - get center
    x = (row['xmin'] + row['xmax']) / 2
    y = (row['ymin'] + row['ymax']) / 2
    
    # main key for this dict will be by floor
    level = row['level']
    if not level in ap_dict:
        ap_dict[level] = {}
    
    # augment scan data with positional data for each scanned AP
    ap_dict[level][(x, y)] = compare_networks(json.loads(row['data']), level)
    
# sort based on floor, and display end message
ap_dict = dict(sorted(ap_dict.items()))
print(f"Dict created, with {len(ap_dict)} keys with the following value lengths: \
{[len(val) for val in ap_dict.values()]}\n")

for level, level_data in ap_dict.items():
    print(f"Level {level}")
    for geometry, scan in level_data.items():
        print(f"{geometry} := net count/scan: {[len(s) for s in scan]}")

In [ ]:
# copy paste of the EXACT trilateration algo used in jsgeodesy
# the python implementation is different! so we use this one

point_nvectors = {}
def trilaterate(point1, distance1, point2, distance2, point3, distance3, radius=6371e3, point_nvectors=point_nvectors, Nvector=Nvector):
        # from en.wikipedia.org/wiki/Trilateration
        
        
        try:
            n1 = point_nvectors[(point1.lat, point1.lon)]
        except KeyError:
            n1 = point1.toNvector()
            point_nvectors[(point1.lat, point1.lon)] = n1
        
        try:
            n2 = point_nvectors[(point2.lat, point2.lon)]
        except KeyError:
            n2 = point2.toNvector()
            point_nvectors[(point2.lat, point2.lon)] = n2
        
        try:
            n3 = point_nvectors[(point3.lat, point3.lon)]
        except KeyError:
            n3 = point3.toNvector()
            point_nvectors[(point3.lat, point3.lon)] = n3
        
        δ1 = distance1/radius
        δ2 = distance2/radius
        δ3 = distance3/radius

        # the following uses x,y coordinate system with origin at n1, x axis n1->n2
        eX = n2.minus(n1).unit()                         # unit vector in x direction n1->n2
        i = eX.dot(n3.minus(n1))                         # signed magnitude of x component of n1->n3
        eY = n3.minus(n1).minus(eX.times(i)).unit()      # unit vector in y direction
        d = n2.minus(n1).length                          # distance n1->n2
        j = eY.dot(n3.minus(n1))                         # signed magnitude of y component of n1->n3
        x = (δ1*δ1 - δ2*δ2 + d*d) / (2*d)                # x component of n1 -> intersection
        y = (δ1*δ1 - δ3*δ3 + i*i + j*j) / (2*j) - x*i/j  # y component of n1 -> intersection
        # const eZ = eX.cross(eY);                            # unit vector perpendicular to plane
        # const z = Math.sqrt(δ1*δ1 - x*x - y*y);             # z will be NaN for no intersections

        n = n1.plus(eX.times(x)).plus(eY.times(y)) # note don't use z component; assume points at same height

        return Nvector(n.x, n.y, n.z).toLatLon()

In [7]:
# the formula we're tuning!
# rssiToDistance in Trilateration.js
def rssi_to_distance(A, n, RSSI, math=math):
    dist = math.pow(10, ((RSSI - A) / (-10 * n)))
    return dist

# trilaterate in Trilateration.js
def do_trilaterate(A, n, triplet, rssi_to_distance=rssi_to_distance, trilaterate=trilaterate):
    
    #points = [LatLon(net['coordinates'][0], net['coordinates'][1]) for net in triplet]
    points = [net['coordinates'] for net in triplet]
    distances = [rssi_to_distance(A, n, net['RSSI']) for net in triplet]
    
    try:
        point = trilaterate(
            points[0],
            distances[0],
            points[1],
            distances[1],
            points[2],
            distances[2]
        )
        
        pointArr = [point.lat, point.lon]
    
    except Exception as e:
        print(e)
        pointArr = [-1, -1]
        
    return pointArr

def distance(point1, point2, math=math):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# distance in Trilateration.js
def meter_distance(point1, point2):
    return point1.distanceTo(point2)
    #return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

# getStats in Trilateration.js
def get_stats(data, distance=distance, math=math):
    pointSum = [sum(i) for i in zip(*data)]
    
    avg = [pointSum[0] / len(data), pointSum[1] / len(data)]
    sumErrSq = sum([distance(point, avg) ** 2 for point in data])
    
    variance = sumErrSq / len(data)
    sd = math.sqrt(variance)
    
    return avg, sd


def iterate_all(A, n, networks, do_trilaterate=do_trilaterate, get_stats=get_stats, distance=distance):
    
    allPoints = []
    
    for i in range(len(networks) - 2):
        for j in range(i + 1, len(networks) - 1):
            for k in range(j + 1, len(networks)):
                triplet = [networks[i], networks[j], networks[k]]
                data = do_trilaterate(A, n, triplet)
                
                if data[0] != -1:
                    allPoints.append(data)
                    
    sdCount = 2
    pointDifference = 999

    while pointDifference != 0:
        originalPointCount = len(allPoints)
        avg, sd = get_stats(allPoints)
        
        newPoints = [point for point in allPoints if distance(avg, point) < sdCount * sd]
        pointDifference = originalPointCount - len(newPoints)
        
        if pointDifference != len(allPoints):
            allPoints = newPoints
        else:
            pointDifference = 0
            
    pointSum = [sum(i) for i in zip(*allPoints)]
    
    return [pointSum[0] / len(allPoints), pointSum[1] / len(allPoints)]
                    

In [8]:
# gets and processes error for the chosen scan location/floor/all data
def err_controller(A, n, ap_dict=ap_dict, iterate_all=iterate_all, meter_distance=meter_distance, LatLon=LatLon):
    err_bound = (0, 10)
    error_per_point = []
    levels = [2]
    
    lat_lons = {}
    
    # for each level we want to do this on...
    for level in levels:
        level_data = ap_dict[level]
        
        # for each sample location... (10 per floor 0:3, 6 on floor 4, 3 on floor 5)        
        for actual_point, scans in level_data.items():
            actual_point = tuple(actual_point)
            if actual_point in lat_lons:
                actual_point_ll = lat_lons[actual_point]
            else:
                actual_point_ll = LatLon(*actual_point)
                lat_lons[actual_point] = actual_point_ll
            
            # for each of the 10 scans in that location...
            err_total = 0.0
            
            for scan in scans:
                
                # get the calculated location using trilat, and get dist to actual point
                predicted_point = iterate_all(A, n, scan)
                predicted_point = tuple(predicted_point)
                
                if predicted_point in lat_lons:
                    predicted_point_ll = lat_lons[predicted_point]
                else:
                    predicted_point_ll = LatLon(*predicted_point)
                    lat_lons[predicted_point] = predicted_point_ll
                
                err = meter_distance(predicted_point_ll, actual_point_ll)
                
                # error function?
                err_total += err
            
            # average over each scan
            err_total = err_total / len(scans)
            error_per_point.append(err_total)
                
    err_avg = sum(error_per_point) / len(error_per_point)
    
    if err_avg < err_bound[0]:
        err_avg = err_bound[0]
    if err_avg > err_bound[1]:
        err_avg = err_bound[1]
    
    return (err_avg, error_per_point)

In [9]:
# function that obtains all the Z axis data, for a given A/n
def f_trilat_pool(A_space, n_space):
    
    Z = []
    min_err = {'err': 9999, 'A': -1, 'n': -1, 'errors': []}
    for A_i, A in enumerate(A_space):
        
        print(f"Working on A {A_i+1}/{A_div} - Executing processes...")
        
        with Pool() as pool:
            data = pool.starmap(err_controller, zip(repeat(A), n_space))
        
        Z_sub = [tup[0] for tup in data]        
        
        print(f"Processes concluded.") 

        for idx, err in enumerate(Z_sub):
            if err < min_err['err']:
                min_err['err'] = err
                min_err['A'] = A
                min_err['n'] = n_space[idx]
                min_err['errors'] = data[idx][1]
        
        Z.append(Z_sub)
        
        
    print(f"\nMin err of {min_err['err']} at (A: {min_err['A']}; n: {min_err['n']})")
    print(f"Error at each scan: {[round(err, 1) for err in min_err['errors']]}")
    
    return np.array(Z).transpose()

In [ ]:
A_div = 64
n_div = 64
A = np.linspace(50, -100, A_div)
n = np.linspace(2, 10, n_div)

toc = timeit.default_timer()
Z = f_trilat_pool(A, n)
tic = timeit.default_timer()

print(f"Time taken to process: {tic - toc}s")

A, n = np.meshgrid(A, n)
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.set_xlabel("A")
ax.set_ylabel("n")
ax.set_zlabel("Error (m)")
ax.plot_surface(A, n, Z, rstride=1, cstride=1, cmap='viridis', edgecolor='none')

plt.show()

Working on A 1/64 - Executing processes...
Processes concluded.
Working on A 2/64 - Executing processes...
Processes concluded.
Working on A 3/64 - Executing processes...
Processes concluded.
Working on A 4/64 - Executing processes...
Processes concluded.
Working on A 5/64 - Executing processes...
Processes concluded.
Working on A 6/64 - Executing processes...
Processes concluded.
Working on A 7/64 - Executing processes...
Processes concluded.
Working on A 8/64 - Executing processes...
Processes concluded.
Working on A 9/64 - Executing processes...
Processes concluded.
Working on A 10/64 - Executing processes...
Processes concluded.
Working on A 11/64 - Executing processes...
Processes concluded.
Working on A 12/64 - Executing processes...
Processes concluded.
Working on A 13/64 - Executing processes...
Processes concluded.
Working on A 14/64 - Executing processes...
Processes concluded.
Working on A 15/64 - Executing processes...
Processes concluded.
Working on A 16/64 - Executing pro